In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math
from config import *

In [3]:
DATA_PATH = "predictions/preds_next_season.csv"

In [4]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [5]:
df.columns

Index(['Name_original', 'GW', 'Season', 'Team', 'Opponent', 'Was_home',
       'Team_rating', 'Opp_rating', 'Price', 'FPL_pos', 'FPL_points',
       'Prediction'],
      dtype='object')

In [6]:
df = df[df["GW"] == NEXT_GAMEWEEK]

In [7]:
df.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
4,Erling Haaland,6.0,2023-24,Manchester City,Nott'ham Forest,1.0,2087.353760,1687.359619,14.1,FWD,NaN,8.773204
78,Mohamed Salah,6.0,2023-24,Liverpool,West Ham,1.0,1961.337402,1807.522339,12.5,MID,NaN,6.774089
117,James Maddison,6.0,2023-24,Tottenham,Arsenal,0.0,1846.944458,1928.752808,7.8,MID,NaN,6.306911
162,James Ward-Prowse,6.0,2023-24,West Ham,Liverpool,0.0,1807.522339,1961.337402,6.1,MID,NaN,5.854319
175,Bryan Mbeumo,6.0,2023-24,Brentford,Everton,1.0,1826.335205,1682.138794,6.8,MID,NaN,5.783767
203,Bukayo Saka,6.0,2023-24,Arsenal,Tottenham,1.0,1928.752808,1846.944458,8.7,MID,NaN,5.615363
207,Bruno Borges Fernandes,6.0,2023-24,Manchester Utd,Burnley,0.0,1860.949707,1702.987793,8.4,MID,NaN,5.601026
257,Solly March,6.0,2023-24,Brighton,Bournemouth,1.0,1839.788574,1659.008667,6.6,MID,NaN,5.433329
317,Carlton Morris,6.0,2023-24,Luton,Wolves,1.0,1592.930542,1708.177246,5.5,FWD,NaN,5.273568
402,Destiny Udogie,6.0,2023-24,Tottenham,Arsenal,0.0,1846.944458,1928.752808,4.7,DEF,NaN,5.045279


In [8]:
# df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [9]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

((500, 12), (59, 12), (172, 12), (213, 12), (56, 12))

In [10]:
df.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
4,Erling Haaland,6.0,2023-24,Manchester City,Nott'ham Forest,1.0,2087.353760,1687.359619,14.1,FWD,NaN,8.773204
78,Mohamed Salah,6.0,2023-24,Liverpool,West Ham,1.0,1961.337402,1807.522339,12.5,MID,NaN,6.774089
117,James Maddison,6.0,2023-24,Tottenham,Arsenal,0.0,1846.944458,1928.752808,7.8,MID,NaN,6.306911
162,James Ward-Prowse,6.0,2023-24,West Ham,Liverpool,0.0,1807.522339,1961.337402,6.1,MID,NaN,5.854319
175,Bryan Mbeumo,6.0,2023-24,Brentford,Everton,1.0,1826.335205,1682.138794,6.8,MID,NaN,5.783767


In [11]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [12]:
df_original = df.copy()

In [13]:
df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [14]:
smallDB = []
for i, row in df.iterrows():
    smallDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [15]:
def find_most_expensive(playersDB):
    index = 0
    price = 0
    for i, p in enumerate(playersDB):
        if p.price > price:
            index = i
            price = p.price
    return index, price
    

In [16]:
def find_N_cheapest_on_position(playersDB, n_players, position):
    output = []
    for p in playersDB:
        if p.position == position:
            if len(output) < n_players:
                output.append(p)
            else:
                i, price = find_most_expensive(output)
                if p.price < price:
                    output[i] = p
    return output

In [17]:
find_N_cheapest_on_position(playersDB, 1, "FWD")[0].__str__()

'(Shola Shoretire, FWD, 4.4, 0.5, Manchester Utd) - (fixtures: 1.0)'

In [18]:
# for p in playersDB:
#     print(p.__str__())

In [19]:
# 2 example Goalkeepers
# playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
# playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [20]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 2, 3],
    [5, 3, 2],
    [5, 4, 1],
]

In [21]:
budget = 100

In [22]:
def fill_bench(playersDB, formation):
    n_gk, n_def, n_mid, n_fwd = 1, 5 - formation[0], 5 - formation[1], 3 - formation[2]
    gks = find_N_cheapest_on_position(playersDB, n_gk, "GK")
    defs = find_N_cheapest_on_position(playersDB, n_def, "DEF")
    mids = find_N_cheapest_on_position(playersDB, n_mid, "MID")
    fwds = find_N_cheapest_on_position(playersDB, n_fwd, "FWD")
    print(formation, n_gk, n_def, n_mid, n_fwd)
    bench = gks + defs + mids + fwds
    for p in bench:
        print(p.__str__())
        p.benched = True
    bench_price = sum([p.price for p in bench])
    print(bench_price)
    return bench, bench_price

In [23]:
best_teams = []
best_team = None
best_team_score = 0
best_bench = None

for formation in possible_formations:
    bench, bench_price = fill_bench(playersDB, formation)
    best_team_for_formation = best_full_teams(smallDB, [formation], budget - bench_price)
    
    if best_team_for_formation[0][1] > best_team_score:
        best_team = best_team_for_formation
        best_team_score = best_team_for_formation[0][1]
        best_bench = bench
        
for p in best_team[0][2]:
    p.benched = True
for p in best_bench:
    best_team[0][2].append(p)
        
    

[3, 4, 3] 1 2 1 0
(Robin Olsen, GK, 4.0, 0.0, Aston Villa) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 2.04, Brighton) - (fixtures: 1.0)
(Sergio Gómez, DEF, 3.9, 0.92, Manchester City) - (fixtures: 1.0)
(Jakub Moder, MID, 4.4, 2.12, Brighton) - (fixtures: 1.0)
16.200000000000003
With formation [3, 4, 3]: 63.36  | (price = 83.3)
(Guglielmo Vicario, GK, 5.1, 4.8, Tottenham) - (fixtures: 1.0)
(Destiny Udogie, DEF, 4.7, 5.05, Tottenham) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 7.9, 5.0, Liverpool) - (fixtures: 1.0)
(Axel Disasi, DEF, 5.1, 4.95, Chelsea) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.77, Liverpool) - (fixtures: 1.0)
(James Maddison, MID, 7.8, 6.31, Tottenham) - (fixtures: 1.0)
(James Ward-Prowse, MID, 6.1, 5.85, West Ham) - (fixtures: 1.0)
(Bryan Mbeumo, MID, 6.8, 5.78, Brentford) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.1, 8.77, Manchester City) - (fixtures: 1.0)
(Carlton Morris, FWD, 5.5, 5.27, Luton) - (fixtures: 1.0)
(Alexander Isak, FWD, 7.7, 4.81, Newcastle U

In [24]:
for p in best_team[0][2]:
    print(p.__str__())

(Guglielmo Vicario, GK, 5.1, 4.8, Tottenham) - (fixtures: 1.0)
(Destiny Udogie, DEF, 4.7, 5.05, Tottenham) - (fixtures: 1.0)
(Axel Disasi, DEF, 5.1, 4.95, Chelsea) - (fixtures: 1.0)
(Ben Mee, DEF, 4.9, 4.85, Brentford) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.77, Liverpool) - (fixtures: 1.0)
(James Maddison, MID, 7.8, 6.31, Tottenham) - (fixtures: 1.0)
(James Ward-Prowse, MID, 6.1, 5.85, West Ham) - (fixtures: 1.0)
(Bryan Mbeumo, MID, 6.8, 5.78, Brentford) - (fixtures: 1.0)
(Bukayo Saka, MID, 8.7, 5.62, Arsenal) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.1, 8.77, Manchester City) - (fixtures: 1.0)
(Carlton Morris, FWD, 5.5, 5.27, Luton) - (fixtures: 1.0)
(Robin Olsen, GK, 4.0, 0.0, Aston Villa) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 2.04, Brighton) - (fixtures: 1.0)
(Sergio Gómez, DEF, 3.9, 0.92, Manchester City) - (fixtures: 1.0)
(Shola Shoretire, FWD, 4.4, 0.5, Manchester Utd) - (fixtures: 1.0)


In [25]:
%%time
# best_team = best_full_teams(smallDB, possible_formations, budget)

CPU times: total: 0 ns
Wall time: 0 ns


In [26]:
f"Best team score: {best_team[0][1]}"

'Best team score: 64.02000000000001'

In [27]:
best_team[0]

([3, 5, 2],
 64.02000000000001,
  <knapsack_problem.player.Player at 0x13720c33b50>])

In [28]:
def best_team_to_df(best_team_squad, df):
    names = [p.name for p in best_team_squad]
    print(names)
    custom_fpl_order = ["FWD", "MID", "DEF", "GK"]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    output = df[df["Name_original"].isin(names)].reset_index()
    output["First_eleven"] = True
    output.loc[11:15, "First_eleven"] = False
    output = output.sort_values(by=["First_eleven", "FPL_pos", "Price"], ascending=False)
    
    return output

In [29]:
best_squad = best_team_to_df(best_team[0][2], df_original)
best_squad

['Guglielmo Vicario', 'Destiny Udogie', 'Axel Disasi', 'Ben Mee', 'Mohamed Salah', 'James Maddison', 'James Ward-Prowse', 'Bryan Mbeumo', 'Bukayo Saka', 'Erling Haaland', 'Carlton Morris', 'Robin Olsen', 'Tariq Lamptey', 'Sergio Gómez', 'Shola Shoretire']


,index,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction,First_eleven
10,537,Guglielmo Vicario,6.0,2023-24,Tottenham,Arsenal,0.0,1846.944458,1928.752808,5.1,GK,NaN,4.796574,True
8,440,Axel Disasi,6.0,2023-24,Chelsea,Aston Villa,1.0,1774.226440,1828.641479,5.1,DEF,NaN,4.947014,True
9,500,Ben Mee,6.0,2023-24,Brentford,Everton,1.0,1826.335205,1682.138794,4.9,DEF,NaN,4.850870,True
7,402,Destiny Udogie,6.0,2023-24,Tottenham,Arsenal,0.0,1846.944458,1928.752808,4.7,DEF,NaN,5.045279,True
1,78,Mohamed Salah,6.0,2023-24,Liverpool,West Ham,1.0,1961.337402,1807.522339,12.5,MID,NaN,6.774089,True
5,203,Bukayo Saka,6.0,2023-24,Arsenal,Tottenham,1.0,1928.752808,1846.944458,8.7,MID,NaN,5.615363,True
2,117,James Maddison,6.0,2023-24,Tottenham,Arsenal,0.0,1846.944458,1928.752808,7.8,MID,NaN,6.306911,True
4,175,Bryan Mbeumo,6.0,2023-24,Brentford,Everton,1.0,1826.335205,1682.138794,6.8,MID,NaN,5.783767,True
3,162,James Ward-Prowse,6.0,2023-24,West Ham,Liverpool,0.0,1807.522339,1961.337402,6.1,MID,NaN,5.854319,True
0,4,Erling Haaland,6.0,2023-24,Manchester City,Nott'ham Forest,1.0,2087.353760,1687.359619,14.1,FWD,NaN,8.773204,True


In [30]:
best_squad.to_csv("predictions/best_squad.csv")